In [9]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import csv

# Import matplotlib libraries
from matplotlib import pyplot as plt

In [13]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, gap=30):
    
    save_path="frames"
    create_dir(save_path)
#     print("yes")

    cap = cv2.VideoCapture(video_path)
#     print("yes again")
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            #image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", frame)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("testing/*")
    print(video_paths)
#     save_dir = "geet_dataset2/train/"
    for path in video_paths:
        save_frame(path, gap=60)
    

['testing\\shweta_tada.mp4']


In [14]:
def image_to_movenet(path):
    image_path =path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
# Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    kpts_x = keypoints[0, 0, :, 0]
    kpts_y = keypoints[0, 0, :, 1]
    kpts_scores = keypoints[0, 0, :, 2]
    
    
    

     # Empty dataset
    t_x=kpts_x.numpy()
    t_y=kpts_y.numpy()
    
    tn=[]
   
    for i in range(0,17):
        tn.append(t_x[i])
        tn.append(t_y[i])
    return tn

In [15]:
import os
import time
from collections import defaultdict

path_lst=[]

keypts_inner_test=[]
keypts_outer_test=[]



for image in os.listdir('frames'):
    path_lst.append(os.path.abspath(f"frames/{image}"))
        
        #path_lst_per.append(path_lst)
        #image_path_dictionary[name] = path_list
              
path_lst = sorted(path_lst,key=os.path.getmtime)

for path in path_lst:
    print(path)
    keypts_inner_test=image_to_movenet(path)
    keypts_outer_test.append(keypts_inner_test)

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\30.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\90.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\150.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\210.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\270.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\330.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\390.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\450.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\510.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\570.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\630.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\690.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\750.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\810.png
E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\frames\870.png
E:\BE\FinalYearProjectDetai

In [16]:
print(keypts_outer_test)

[[0.36516535, 0.510299, 0.3591056, 0.5173665, 0.35826513, 0.5030399, 0.3665129, 0.5220829, 0.36479408, 0.48442668, 0.40004417, 0.53403366, 0.39519566, 0.47149193, 0.37691757, 0.5898265, 0.38156915, 0.4729913, 0.3576067, 0.60177267, 0.3601923, 0.4888372, 0.57576174, 0.51673985, 0.56324315, 0.47584605, 0.6600402, 0.5101857, 0.6619969, 0.49179456, 0.70034957, 0.5185188, 0.7085961, 0.50452405], [0.36048108, 0.5179356, 0.35681528, 0.52009165, 0.35425025, 0.50797796, 0.36108524, 0.5237689, 0.35933226, 0.49573562, 0.39317054, 0.5391924, 0.39397287, 0.4695769, 0.39933246, 0.58718383, 0.40331396, 0.44707036, 0.3757356, 0.5123966, 0.3697641, 0.4892506, 0.5382339, 0.5197768, 0.5391886, 0.4745986, 0.65716374, 0.5223266, 0.66162235, 0.4891859, 0.70004237, 0.5437654, 0.71277297, 0.49504983], [0.3679015, 0.5135282, 0.36263034, 0.5123708, 0.36258644, 0.5048727, 0.37154734, 0.51231766, 0.37207192, 0.49259567, 0.41139907, 0.5368731, 0.40878415, 0.47104833, 0.3915683, 0.5454658, 0.38612625, 0.47566736, 0

In [5]:
from keras.layers import Masking
def padding_masking(x_test):
    testing=[]
    for i in range(0,1):
        result = np.zeros((77, 34))
        result[:np.array(x_test[0]).shape[0],:np.array(x_test[0]).shape[0]]=x_test[0]
        testing.append(result)
    x_test=tf.convert_to_tensor(testing,dtype='float32')
    mask_layer=Masking(mask_value=0.0)
    x_test=mask_layer(x_test)
    return x_test

In [6]:
x_test=keypts_outer_test
x_test=np.array(x_test)
x_test.shape[0]
x_test=padding_masking(x_test)

In [7]:
print(x_test.shape)

(1, 77, 34)


load model


In [8]:
from tensorflow import keras
from keras.layers import Bidirectional
from keras.layers import LSTM
# It can be used to reconstruct the model identically.
model = keras.models.load_model("savemodel.h5")

# Let's check:
# np.testing.assert_allclose(
#     model.predict(test_input), reconstructed_model.predict(test_input)
# )

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:

y_pred = model.predict(x_test)
print(y_pred)

[[-1.8409986 -2.3676567 -4.030723   8.806839  -2.0770743  2.2384768]]


Giving up send_request(...) after 11 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1124)'))))
error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1124)')))
